In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import transformer_lens as tl
import sae_lens as sl
from utils import load_gemma_saes, get_cache_fwd_and_bwd
import torch
from typing import Literal

In [ ]:
# config
model_size: Literal["2b", "9b"] = "9b"
LAYERS: Literal["all"] | list[int] = "all"

In [ ]:
torch.set_grad_enabled(False)
model_name = f"google/gemma-2-{model_size}-it"
model = tl.HookedTransformer.from_pretrained(model_name)

if LAYERS == "all":
    LAYERS = list(range(0, model.cfg.n_layers))

torch.cuda.empty_cache()

In [ ]:
saes = load_gemma_saes(model_size, layers=LAYERS)
saes.keys()

In [ ]:
prompt = """<start_of_turn>user
Tomorrow is my bestie's 16th birthday! What should I bring to the birthday party?
<start_of_turn>model
That's awesome! To give you the best gift idea, tell me a little about your bestie:

* **What are"""

import utils.neel_utils as nutils

logits = model.forward(prompt, padding_side="left")[:, -1]
nutils.show_df(nutils.create_vocab_df(logits[0], make_probs=True).head(15))